Workspace file to test new codes, debug, troubleshoot.

In [1]:
#!/usr/bin/env python3

"""This demo script simultaneously displays the koi fish video will writing haptic output
to the haptic display via USB. Both the video and haptic output will loop continuously
until the user presses the 'B' keyboard key.

To run, it requires both the input video and the output.txt file generated by the
visual-haptic algorithm preprocessing code."""

###### USER SETTINGS ######
FILENAME = "algo_input_data/datakoi_output_scaled.txt"
VIDEONAME = "algo_input_videos/video_koi.mp4"
SERIAL_ACTIVE = False # if False, just runs the algorithm without sending to HV switches
COM_A = "COM9" # port for MINI switches 1-10
COM_B = "COM14" # port for MINI switches 11-20
COM_C = "COM15" # port for MINI swiches 21-28

###### INITIALIZATIONS ######
import cv2
import time
import haptic_utils.haptic_map as haptic_map # my custom file
import haptic_utils.USB as USB # my custom file
from numpy import genfromtxt

###### MAIN ######
# Load data
data=genfromtxt(FILENAME,delimiter=',')[:,0:28]
data_length = data.shape[0]
frame_rate = 20

# Set up USBWriter and intensity map:
serial_ports = [COM_A, COM_B, COM_C]
serial_writer = USB.SerialWriter(serial_ports, serial_active=SERIAL_ACTIVE)

# Preprocess data:
output_data = haptic_map.make_output_data(data,
                                        freq_range=(5,24),
                                        duty_range=(0.5,0.2))

# Preprocess video:
video_data = []
cap = cv2.VideoCapture(VIDEONAME)
while True:
    ret,img = cap.read()
    if ret:
        video_data.append(img)
    else:
        break
video_data.reverse()


In [8]:
len(video_data)

1799

In [23]:
data2 = data.reshape((1799,4,7))
data3=[]
for array in data2:
    data3.append(array)

In [27]:
data3[3]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.01275722],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])